In [19]:
import io
import fitz
import re
from PIL import Image, ImageDraw, ImageFont, ImageColor
import operator
import pandas as pd 
import numpy as np
from datetime import datetime
import math
from tqdm import tqdm

# Ploting funcs

In [20]:
def plot_genus_blocks(page_df, draw, color = '#6c899e', w = 3):
    try:
        genus_list = page_df['draw_genus'].unique()
    except:
        print("no GENUS found")
        return 

    for g in genus_list:
        temp_df = page_df[(page_df['draw_genus'] == g)]
        g_x0 = temp_df['x0'].min()
        g_y0 = temp_df['y0'].min()
        g_x1 = temp_df['x1'].max()
        g_y1 = temp_df['y1'].max()

        draw.rectangle((g_x0, g_y0, g_x1, g_y1), fill=None, outline=ImageColor.getrgb(color), width = w)
        
def plot_epithet_blocks(page_df, draw, color = '#660066', w = 3):
    try:
        epithet_list = page_df['draw_epithet'].unique()
    except:
        print("no EPITHET found")
        return 
    
    for e in epithet_list:
        temp_df = page_df[(page_df['draw_epithet'] == e)]
        e_x0 = temp_df['x0'].min()
        e_y0 = temp_df['y0'].min()
        e_x1 = temp_df['x1'].max()
        e_y1 = temp_df['y1'].max()

        draw.rectangle((e_x0, e_y0, e_x1, e_y1), fill=None, outline=ImageColor.getrgb(color), width = w)

def plot_author_blocks(page_df, draw, color = '#a3a3a3', w = 1):
    try:
        author_list = page_df['draw_author'].unique()
    except:
        print("no AUTHOR found")
        return 

    for a in author_list:
        temp_df = page_df[(page_df['draw_author'] == a)]
        e_x0 = temp_df['x0'].min()
        e_y0 = temp_df['y0'].min()
        e_x1 = temp_df['x1'].max()
        e_y1 = temp_df['y1'].max()

        draw.rectangle((e_x0, e_y0, e_x1, e_y1), fill=None, outline=ImageColor.getrgb(color), width = w)

def plot_infra_blocks(page_df, draw, color = '#ff6289', w = 1):
    try:
        infra_list = page_df['draw_infra'].unique()
    except:
        print("no INFRA Spp. found")
        return 

    for infra_spp in infra_list:
        temp_df = page_df[(page_df['draw_infra'] == infra_spp)]
        e_x0 = temp_df['x0'].min()
        e_y0 = temp_df['y0'].min()
        e_x1 = temp_df['x1'].max()
        e_y1 = temp_df['y1'].max()

        draw.rectangle((e_x0, e_y0, e_x1, e_y1), fill=None, outline=ImageColor.getrgb(color), width = w)

def plot_valid_words(page_df, draw, color = '#660044', w = 2):
    blocks = page_df['block_no'].unique()
    """for b in blocks:
        lines = page_df[page_df['block_no'] == b]['line_no'].unique()
        for l in lines:
            cond = (page_df['line_no'] == l) & (page_df['block_no'] == b)
            words = page_df[cond]['word_no'].unique()
            page_df = page_df.copy()
            for w in words:
                x0 = page_df[(cond) & (page_df['word_no'] == w)]['x0'].item()
                y0 = page_df[(cond) & (page_df['word_no'] == w)]['y0'].item()
                x1 = page_df[(cond) & (page_df['word_no'] == w)]['x1'].item()
                y1 = page_df[(cond) & (page_df['word_no'] == w)]['y1'].item()
                draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb(color), width = w)
    """
    for index, row in page_df.iterrows():
        x0, y0, x1, y1 = row['x0'], row['y0'], row['x1'], row['y1'] 
        draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb(color), width = w)

# Import Vol3 Index

In [21]:
#pdf_dir = "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf"
pdf_dir = "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf"
index = range(616, 639)
doc = fitz.open(pdf_dir)
pages = [doc[i] for i in range(doc.pageCount)]
#index = list(range(555, 583))

pdf_dir = "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf"
index = range(616, 639)

TARGET_DPI = 300
mat = fitz.Matrix(TARGET_DPI/ 72, TARGET_DPI/ 72)

indent_groups = []
indent_err = 15

# regex based boolean functions

In [41]:
def valid(word):
    """
    valid words are words that are:
    - at least 2 characters
        - unless it's x (symbol for hybrid)
    """
    return (not bool(re.search(r"[0-9]+[,.]?", word))) and \
            (word != 'NOUVELLE' and word != 'FLORE') and \
            (len(word) > 1 or \
                word == 'x' or word == 'X' or word == '×' or word == r'\u00D7') and \
            ''.join(e for e in word if e.isalpha()).isalpha()
    
def is_genus(word):
    """
    A word in the index might be a genus if it satisfies the following properties:
    - letters: french alphabet + at most one hyphen (which is not first or last letter)
        - first letter upper case
        - all but first lowecase 
    in regex: ^[A-ZÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ]{1}[a-zàâäèéêëîïôœùûüÿç]*[-]?[a-zàâäèéêëîïôœùûüÿç]+$ #ignoring strict beggining and end cause of noise
        * based on the current expression it'd also be at least 2 letters long
    """
    regex = r"[A-ZÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ\u00D7]{1}[a-zàâäèéêëîïôœùûüÿç]*[-]?[a-zàâäèéêëîïôœùûüÿç]+"
    return re.search(regex, word)
    

def is_epithet(word):
    """
    A word in the index might be an epithet if it satisfies the following properties:
    - letters: french alphabet + at most one hyphen (which is not first or last letter)
        - all letters lowecase 
    in regex: ^[a-zàâäèéêëîïôœùûüÿç]+[-]?[a-zàâäèéêëîïôœùûüÿç]+$ #ignoring strict beggining and end cause of noise 
        * based on the current expression it'd also be at least 2 letters long
    """
    regex = r"[a-zàâäèéêëîïôœùûüÿç\u00D7]+[-]?[a-zàâäèéêëîïôœùûüÿç]+"
    return re.search(regex, word)
    
def is_hybrid(word):
    regex = r"^(([Xx\u00D7])|([Xx\u00D7]\.))$"
    return re.search(regex, word)

def is_infra(word):
    regex = r"^(var\.)|(subsp\.)"
    return re.search(regex, word)

# pre-processing func

In [114]:
def preprocessing(page_num, indent_err = 30):
    
    #initiate dataframe
    page_df = pd.DataFrame(pages[page_num].get_text_words(), columns =['in_x0', 'in_y0', 'in_x1', 'in_y1', 'word', 'block_no', 'line_no', 'word_no'])
    
    #add page number to dataframe
    page_df['page_num'] = np.array([page_num]*page_df.shape[0])
    #initiate all columns that will be added
    page_df['genus'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_genus'] = np.array([np.NaN]*page_df.shape[0])
    page_df['epithet'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_epithet'] = np.array([np.NaN]*page_df.shape[0])
    page_df['author'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_author'] = np.array([np.NaN]*page_df.shape[0])
    page_df['infra'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_infra'] = np.array([np.NaN]*page_df.shape[0])
    page_df['taxon rank'] = np.array([np.NaN]*page_df.shape[0])
    page_df['error_check'] = np.array([np.NaN]*page_df.shape[0])
    #updating coordinates to represent target DPI
    page_df['x0'], page_df['y0'], page_df['x1'], page_df['y1'] = page_df['in_x0']*TARGET_DPI/ 72, page_df['in_y0']*TARGET_DPI/ 72, page_df['in_x1']*TARGET_DPI/ 72, page_df['in_y1']*TARGET_DPI/ 72
    #get x corner coordinates 
    x_min = page_df['x0'].min()
    x_max = page_df['x1'].max()

    y_max = page_df['y1'].max()

    #Remove the extra flore - 18 at page 545
    if page_num == index[4]:
        page_df = page_df[~((page_df["word"] == 'Flore') & (page_df['y1'] == y_max))]
    #invalid words dataframe -- for error checking
    pruned_words_df = page_df[~page_df["word"].apply(valid)].reset_index()
    #prune out invalid words (based on function valid)
    page_df = page_df[page_df["word"].apply(valid)].reset_index()
    
    indent_groups = []
    blocks = page_df['block_no'].unique()
    for b in blocks:
        lines = page_df[page_df['block_no'] == b]['line_no'].unique()
        for l in lines:
            #reset word_no values (useful for cases where word that was originally at 0th index was pruned out)
            cond = (page_df['line_no'] == l) & (page_df['block_no'] == b)
            num_words = len(page_df[cond]['word_no'])
            page_df.loc[cond, 'word_no'] = np.arange(num_words).astype(int) #this is slowww
            #set column number (0 or 1)
            x_0 = page_df[cond]['x0'].min()
            #THIS DOESN'T WORK AAAA -- issue was with line no thing
            if not np.isnan(x_0):
                page_df.loc[cond, 'col_no'] = np.array([int(x_0 > ((x_min + x_max) / 2))]*num_words).astype(int)

                #initiate indent groups -- only first word should get an indent_group value 
                new_group = True
                for g_i in range(len(indent_groups)):
                    g = indent_groups[g_i]
                    g_arr = np.array(g)
                    if x_0 <= np.mean(g_arr) + indent_err and x_0 >= np.mean(g_arr) - indent_err:
                        g.append(x_0)
                        new_group = False
                        page_df.loc[cond, 'indent_group'] = np.array([g_i]*num_words).astype(int)
                if new_group:
                    indent_groups.append([x_0])
                    g_i = len(indent_groups) - 1
                    page_df.loc[cond, 'indent_group'] = np.array([g_i]*num_words).astype(int)

    #print("indent groups:", indent_groups)
    #return updated page_df, pruned_words_df, indent groups
    return page_df.reset_index(), pruned_words_df, indent_groups

#https://stackoverflow.com/questions/53468558/adding-image-to-pandas-dataframe

In [115]:
genus = np.NaN
df_dict = {}
pruned_dict = {}

for page_num in tqdm(index):
    page_df, pruned_df, indent_group = preprocessing(page_num)
    df_dict[page_num] = page_df
    pruned_dict[page_num] = pruned_df

100%|██████████| 23/23 [00:08<00:00,  2.69it/s]


In [116]:
genus = np.NaN
epithet = np.NaN
draw_genus = np.NaN
draw_epithet = np.NaN
result_ims = []
df_list = []

for page_num in tqdm(index):
    #page_num = index[-1]
    #process the pre-processed dfs
    page_df = df_dict[page_num]
    
    #for drawing
    pix_map = doc.get_page_pixmap(page_num,matrix=mat)
    image = Image.open(io.BytesIO(pix_map.tobytes()))
    draw = ImageDraw.Draw(image)
    
    plot_valid_words(page_df, draw, color = '#660044', w = 2)
    result_ims.append(image)
    
    #break 
TIME_STR = datetime.now().strftime("%Y_%m_%d-%I_%M_%p")
result_ims[0].save('../output/index/PDF/vol1_index_valid_words'+TIME_STR+'.pdf',save_all=True, append_images=result_ims[1:])


100%|██████████| 23/23 [00:06<00:00,  3.70it/s]


# Finding indentations associated with genus, epithet, infra

In [117]:
types = ['genus', 'epithet', 'infra', 'author', 'misc.']
def n_leftmost_indent(df, n):
    """return a tuple with at most 3 elements each element itself is a tuple containing indent group, mean, group len"""
    indent_groups = [(g, df[(df['indent_group'] == g) & (df['word_no'] == 0)]['x0'].mean(), len(df[(df['indent_group'] == g) & (df['word_no'] == 0)]['x0'])) for g in df['indent_group'].unique()]
    indent_groups.sort(key = lambda x : x[1])
    print(indent_groups[:n])
    return indent_groups[:n]

In [118]:
def get_genusEpithetInfra_indent(col_df):
    leftmost_3_indents = n_leftmost_indent(col_df, 2) 
    min_gap = 0
    max_gap = 75

    # possibly not specific enough
    # first identifying indent based don distance from one another only
    """if len(leftmost_3_indents) == 3:
        if leftmost_3_indents[0][1] < max_gap:
            leftmost_3_indents = leftmost_3_indents[1:]
        elif ((leftmost_3_indents[1][1] - leftmost_3_indents[0][1]) > max_gap or \
            (leftmost_3_indents[1][1] - leftmost_3_indents[0][1]) < min_gap): #comparing first two (if satisfied last two will be checked in next if block)
            leftmost_3_indents = [max(leftmost_3_indents[1:], key = lambda x : x[2])] + [leftmost_3_indents[2]]
        elif (leftmost_3_indents[2][1] - leftmost_3_indents[1][1]) > max_gap or \
            (leftmost_3_indents[2][1] - leftmost_3_indents[1][1]) < min_gap: #comparing last two
            leftmost_3_indents = [leftmost_3_indents[0]] + [max(leftmost_3_indents[1:], key = lambda x : x[2])]

    if len(leftmost_3_indents) == 2:
        if leftmost_3_indents[0][1] < max_gap:
            leftmost_3_indents = leftmost_3_indents[1]
        elif (leftmost_3_indents[1][1] - leftmost_3_indents[0][1]) > max_gap or (leftmost_3_indents[1][1] - leftmost_3_indents[0][1]) < min_gap:
            leftmost_3_indents = [max(leftmost_3_indents, key = lambda x : x[2])]"""

    has_genus, has_epithet, has_infra = False, False, False
    genus_indent, epithet_indent, infra_indent = -1, -1, -1
    if len(leftmost_3_indents) == 3 and type(leftmost_3_indents) == type([1,2,3]):
        has_genus, has_epithet, has_infra = True, True, True
        print("leftmost 3:", leftmost_3_indents)
        genus_indent, epithet_indent, infra_indent = [el[0] for el in leftmost_3_indents]
    elif len(leftmost_3_indents) == 2:
        if col_df[col_df['indent_group'] == leftmost_3_indents[1][0]]['word'].apply(is_infra).any():
            has_genus, has_epithet, has_infra = False, True, True
            epithet_indent, infra_indent = [el[0] for el in leftmost_3_indents]
        else:
            has_genus, has_epithet, has_infra = True, True, False
            genus_indent, epithet_indent = [el[0] for el in leftmost_3_indents]
    elif len(leftmost_3_indents) == 1 or type(leftmost_3_indents) == type((1,2,3)): 
        if type(leftmost_3_indents) == type((1,2,3)):
            leftmost_3_indents = [leftmost_3_indents]
        has_genus, has_epithet, has_infra = False, True, False
        epithet_indent = leftmost_3_indents[0][0]

    return genus_indent, epithet_indent, infra_indent, leftmost_3_indents

# Processing column dataframes


In [119]:
def process_col(col_df, genus, epithet, draw_genus, draw_epithet, draw_infra = np.NaN):
    genus_indent, epithet_indent, infra_indent, indent_3_left = get_genusEpithetInfra_indent(col_df)
    print(genus_indent, epithet_indent, infra_indent, indent_3_left)
    
    blocks = col_df['block_no'].unique()
    start_word_cond = -1 
    author = ''
    #draw_infra = np.NaN
    
    for b in blocks:
        lines = col_df[col_df['block_no'] == b]['line_no'].unique()
        for l in lines:
            cond = (col_df['line_no'] == l) & (col_df['block_no'] == b)
            words = col_df[cond]['word_no'].unique()
            process_hybrid = False
            process_infra = False
            
            col_df = col_df.copy()
            for w in words:
                word_cond = (col_df['line_no'] == l) & (col_df['block_no'] == b) & (col_df['word_no'] == w) 
                word = col_df[word_cond]['word'].item()
                #print(word)

                if w == 0:
                    infra = ''
                    if author != '':
                        col_df.loc[start_word_cond, 'author'] = author
                        author = ''
                    
                    start_word_cond = word_cond
                    start_l = l 
                    start_b = b 

                    indent_group = col_df[word_cond]['indent_group'].item()
                    
                    if is_hybrid(word):
                        process_hybrid = True
                        misc = word
                        author = ''
                        #col_df.loc[start_word_cond, 'misc.'] = misc
                    #now only gotta say INDENT AND satisfies these paterns
                    #print(indent_group, genus_indent)
                    else: 
                        if indent_group == genus_indent:
                            if not ''.join(e for e in word if e.isalpha()).isupper():
                                genus = word
                                misc = ''
                                author = ''
                                infra = ''
                                epithet = ''
                                draw_genus = genus
                                col_df.loc[start_word_cond, 'genus'] = genus
                                col_df.loc[start_word_cond, 'taxon rank'] = 'genus'
                                if not is_genus(word):
                                    col_df.loc[start_word_cond, 'error_check'] = True
                                col_df.loc[word_cond, 'draw_genus'] = draw_genus
                                
                            else: 
                                genus = ''
                                misc = ''
                                author = ''
                                infra = ''
                                epithet = ''
                                draw_genus = np.NaN
                            """
                            def set_genus(df, w0_index, error_check = True):
                                if not ''.join(e for e in word if e.isalpha()).isupper():
                                    genus = word
                                    draw_genus = genus
                                    df.loc[w0_index, 'genus'] = genus
                                    df.loc[w0_index, 'taxon rank'] = 'genus'
                                    col_df.loc[word_cond, 'draw_genus'] = draw_genus"""
                        elif indent_group == epithet_indent and not ''.join(e for e in word if e.isalpha()).isupper():
                            epithet = word
                            misc = ''
                            infra = ''
                            author = ''
                            col_df.loc[start_word_cond, 'genus'] = genus
                            col_df.loc[start_word_cond, 'epithet'] = epithet
                            col_df.loc[start_word_cond, 'taxon rank'] = 'species'
                            if not is_epithet(word):
                                col_df.loc[start_word_cond, 'error_check'] = True
                            draw_epithet = str(genus) + '_' + str(epithet) +'_' + str(start_b) + '_' + str(start_l)
                            col_df.loc[word_cond, 'draw_genus'] = draw_genus
                            col_df.loc[word_cond, 'draw_epithet'] = draw_epithet
                        

                        elif indent_group == infra_indent:
                            process_infra = True
                            misc = word
                            author = ''
                            #col_df.loc[start_word_cond, 'misc.'] = misc
                            if not (is_infra(word) or is_hybrid(word)):
                                col_df.loc[start_word_cond, 'error_check'] = True
                    
                elif process_infra:
                    start_word_cond = word_cond
                    start_l = l 
                    start_b = b 
                    infra = word 
                    col_df.loc[start_word_cond, 'genus'] = genus
                    col_df.loc[start_word_cond, 'epithet'] = epithet
                    col_df.loc[start_word_cond, 'infra'] = infra
                    #col_df.loc[start_word_cond, 'misc.'] = misc
                    col_df.loc[start_word_cond, 'taxon rank'] = misc
                    draw_infra = str(infra) + '_'+str(start_b)+'_'+str(start_l)
                    process_infra = False
                    col_df.loc[word_cond, 'draw_genus'] = draw_genus
                    col_df.loc[word_cond, 'draw_epithet'] = draw_epithet
                    col_df.loc[word_cond, 'draw_infra'] = draw_infra
                    
                elif process_hybrid:
                    start_word_cond = word_cond
                    start_l = l 
                    start_b = b 
                    if indent_group == genus_indent:
                        genus = word
                        epithet = ''
                        infra = ''
                        author = ''
                        draw_genus = genus
                        col_df.loc[start_word_cond, 'genus'] = genus
                        col_df.loc[start_word_cond, 'taxon rank'] = 'genus - hybrid'
                        if not is_genus(word):
                            col_df.loc[start_word_cond, 'error_check'] = True
                        col_df.loc[word_cond, 'draw_genus'] = draw_genus
                            
                    elif indent_group == epithet_indent:
                        epithet = word
                        author = ''
                        infra = ''
                        col_df.loc[start_word_cond, 'genus'] = genus
                        col_df.loc[start_word_cond, 'epithet'] = epithet
                        col_df.loc[start_word_cond, 'taxon rank'] = 'species - hybrid'
                        draw_epithet = str(genus) + '_' + str(epithet) +'_' + str(start_b) + '_' + str(start_l)
                        if not is_epithet(word):
                            col_df.loc[start_word_cond, 'error_check'] = True
                        col_df.loc[word_cond, 'draw_genus'] = draw_genus
                        col_df.loc[word_cond, 'draw_epithet'] = draw_epithet
                    elif indent_group == infra_indent:
                        infra = word
                        col_df.loc[start_word_cond, 'genus'] = genus
                        col_df.loc[start_word_cond, 'epithet'] = epithet
                        col_df.loc[start_word_cond, 'infra'] = infra
                        col_df.loc[start_word_cond, 'taxon rank'] = 'hybrid'
                        col_df.loc[word_cond, 'draw_genus'] = draw_genus
                        col_df.loc[word_cond, 'draw_epithet'] = draw_epithet
                        col_df.loc[word_cond, 'draw_infra'] = draw_infra
                    #col_df.loc[start_word_cond, 'misc.'] = 'x'
                    process_hybrid = False
                else:
                    if w == 1 and epithet == '': 
                        epithet = word
                        misc = ''
                        infra = ''
                        author = ''
                        col_df.loc[start_word_cond, 'genus'] = genus
                        col_df.loc[start_word_cond, 'epithet'] = epithet
                        col_df.loc[start_word_cond, 'taxon rank'] = 'species'
                        if not is_epithet(word):
                            col_df.loc[start_word_cond, 'error_check'] = True
                        draw_epithet = str(genus) + '_' + str(epithet) +'_' + str(start_b) + '_' + str(start_l)
                        col_df.loc[word_cond, 'draw_genus'] = draw_genus
                        col_df.loc[word_cond, 'draw_epithet'] = draw_epithet
                    #elif word == 'FLORE':
                    #    author = ''
                    else:
                        author = author + word + ' '
                        col_df.loc[word_cond, 'draw_author'] = 'author_'+str(start_b)+'_'+str(start_l)
                    #col_df.loc[word_cond, 'draw_genus'] = draw_genus
                    #if epithet:
                    #    col_df.loc[word_cond, 'draw_epithet'] = draw_epithet
                    #if infra: 
                    #    col_df.loc[word_cond, 'draw_infra'] = draw_infra

    #Last author
    if author != '':
        col_df.loc[start_word_cond, 'author'] = author
    
    return col_df, genus, epithet, draw_genus, draw_epithet

In [123]:
def process_col(col_df, genus, epithet, draw_genus, draw_epithet, draw_infra = np.NaN):
    genus_indent, epithet_indent, infra_indent, indent_3_left = get_genusEpithetInfra_indent(col_df)
    print(genus_indent, epithet_indent, infra_indent, indent_3_left)
    
    blocks = col_df['block_no'].unique()
    start_word_cond = -1 
    author = ''
    #draw_infra = np.NaN
    
    col_df = col_df.copy()
    for index, row in col_df.iterrows():
        b, l, w = row['block_no'], row['line_no'], row['word_no']
        word, indent_group = row['word'], row['indent_group']
        row_cond = (col_df['line_no'] == l) & (col_df['block_no'] == b) & (col_df['word_no'] == w) 
        process_hybrid = False
        process_infra = False
        if w == 0: 
            start_word_cond = row_cond
            if indent_group == genus_indent and not ''.join(e for e in word if e.isalpha()).isupper():
                genus = word
                draw_genus = genus
                epithet = ''
                draw_epithet = ''
                author = ''
                misc = ''
                infra = ''
                col_df.loc[start_word_cond, 'genus'] = genus
                col_df.loc[start_word_cond, 'taxon rank'] = 'genus'
                if not is_genus(word):
                    col_df.loc[row_cond, 'error_check'] = True
                col_df.loc[row_cond, 'draw_genus'] = draw_genus
                col_df.loc[row_cond, 'author'] = ''

            elif indent_group == epithet_indent and not ''.join(e for e in word if e.isalpha()).isupper():
                epithet = word
                author = ''
                col_df.loc[row_cond, 'genus'] = genus
                col_df.loc[row_cond, 'epithet'] = epithet
                col_df.loc[row_cond, 'taxon rank'] = 'species'
                if not is_epithet(word):
                    col_df.loc[row_cond, 'error_check'] = True
                draw_epithet = str(genus) + '_' + str(epithet) +'_' + str(b) + '_' + str(l)
                col_df.loc[row_cond, 'draw_genus'] = draw_genus
                col_df.loc[row_cond, 'draw_epithet'] = draw_epithet
                col_df.loc[row_cond, 'author'] = ''
        
        else:
            if w == 1 and epithet == '': 
                epithet = word
                misc = ''
                infra = ''
                author = ''
                start_word_cond = row_cond
                col_df.loc[row_cond, 'genus'] = genus
                col_df.loc[row_cond, 'epithet'] = epithet
                col_df.loc[row_cond, 'taxon rank'] = 'species'
                if not is_epithet(word):
                    col_df.loc[row_cond, 'error_check'] = True
                draw_epithet = str(genus) + '_' + str(epithet) +'_' + str(b) + '_' + str(l)
                col_df.loc[row_cond, 'draw_genus'] = draw_genus
                col_df.loc[row_cond, 'draw_epithet'] = draw_epithet
                col_df.loc[row_cond, 'author'] = ''
            #elif word == 'FLORE':
            #    author = ''
            else:
                #print(col_df.loc[start_word_cond, 'author'])
                """if np.isnan(col_df.loc[start_word_cond, 'author'].item()):
                    author == ''
                    col_df.loc[start_word_cond, 'author'] = ''"""
                curr_author_part = word + ' '
                col_df.loc[start_word_cond, 'author'] += curr_author_part
                col_df.loc[row_cond, 'draw_author'] = 'author_'+str(b)+'_'+str(l)
            #col_df.loc[word_cond, 'draw_genus'] = draw_genus
            #if epithet:
            #    col_df.loc[word_cond, 'draw_epithet'] = draw_epithet
            #if infra: 
            #    col_df.loc[word_cond, 'draw_infra'] = draw_infra"""

    #Last author
    """if author != '':
        col_df.loc[start_word_cond, 'author'] = author"""
                    

    return col_df, genus, epithet, draw_genus, draw_epithet


# the Results

In [124]:
genus = np.NaN
epithet = np.NaN
draw_genus = np.NaN
draw_epithet = np.NaN
result_ims = []
df_list = []

for page_num in tqdm(index):
    #if page_num == index[-2]:
    #    break
    #page_num = index[-1]
    #process the pre-processed dfs
    page_df = df_dict[page_num]
    
    #for drawing
    pix_map = doc.get_page_pixmap(page_num,matrix=mat)
    image = Image.open(io.BytesIO(pix_map.tobytes()))
    draw = ImageDraw.Draw(image)

    #processing each column
    for c in page_df['col_no'].unique():
        col_df = page_df[page_df['col_no'] == c]
        col_df, genus, epithet, draw_genus, draw_epithet = process_col(col_df, genus, epithet, draw_genus, draw_epithet)
        df_list.append(col_df)

        #drawing boxes in each column
        plot_genus_blocks(col_df, draw)
        plot_epithet_blocks(col_df, draw)
        plot_author_blocks(col_df, draw)
        plot_infra_blocks(col_df, draw)

    result_ims.append(image)
    #break 
TIME_STR = datetime.now().strftime("%Y_%m_%d-%I_%M_%p")
result_ims[0].save('../output/index/PDF/vol1_index_reverted_ROI'+TIME_STR+'.pdf',save_all=True, append_images=result_ims[1:])

pre_processed_df = pd.concat([df_dict[k] for k in df_dict], axis = 0)
df = pd.concat(df_list, axis = 0)
df.to_html('../output/index/html/vol1_index_reverted'+TIME_STR+'.html')
pre_processed_df.to_html('../output/index/html/vol1_preprocessed_index'+TIME_STR+'.html')
df.to_csv('../output/index/CSV/vol1_index_reverted'+TIME_STR+'.csv', index = False)

pruned = df[(~df['genus'].isnull())]
pruned = pruned[["page_num", "genus", "epithet", "infra" ,"author", "taxon rank"]]
pruned.to_csv('../output/index/CSV/vol1_index_reverted_pruned'+TIME_STR+'.csv', index = False)
pruned.to_html('../output/index/html/vol1_index_reverted_pruned'+TIME_STR+'.html')

  0%|          | 0/23 [00:00<?, ?it/s]

[(2.0, 116.37499829133353, 8), (1.0, 172.15384744171408, 39)]
2.0 1.0 -1 [(2.0, 116.37499829133353, 8), (1.0, 172.15384744171408, 39)]
[(3.0, 972.6666627106845, 9), (4.0, 1030.1499970753987, 40)]
3.0 4.0 -1 [(3.0, 972.6666627106845, 9), (4.0, 1030.1499970753987, 40)]


  4%|▍         | 1/23 [00:01<00:29,  1.34s/it]

[(1.0, 123.00000190734863, 2), (0.0, 184.4999977520534, 56)]
1.0 0.0 -1 [(1.0, 123.00000190734863, 2), (0.0, 184.4999977520534, 56)]


  9%|▊         | 2/23 [00:02<00:25,  1.21s/it]

...

[(4.0, 940.5833350287545, 12), (3.0, 997.4090922962537, 44)]
4.0 3.0 -1 [(4.0, 940.5833350287545, 12), (3.0, 997.4090922962537, 44)]
[(0.0, 169.77777657685454, 9), (1.0, 222.23404248555497, 47)]
0.0 1.0 -1 [(0.0, 169.77777657685454, 9), (1.0, 222.23404248555497, 47)]
[(4.0, 1029.3999989827475, 15), (3.0, 1083.5791093331798, 36)]

 96%|█████████▌| 22/23 [00:23<00:00,  1.08it/s]


4.0 3.0 -1 [(4.0, 1029.3999989827475, 15), (3.0, 1083.5791093331798, 36)]
[(0.0, 59.99999841054281, 1), (1.0, 115.91666738192242, 12)]
0.0 1.0 -1 [(0.0, 59.99999841054281, 1), (1.0, 115.91666738192242, 12)]


100%|██████████| 23/23 [00:24<00:00,  1.05s/it]

[(2.0, 918.374999364217, 8), (3.0, 973.9999771118164, 2)]
2.0 3.0 -1 [(2.0, 918.374999364217, 8), (3.0, 973.9999771118164, 2)]


In [122]:
#df_list[-1]
#df_dict[index[-1]]
#result_ims[-1]

In [40]:
#result_ims[-1]
"Nées".isalpha()

True